In [1]:
import os,sys
sys.path.append('/home/ubuntu/hacking/projects/jan2021/deep-mediation/src')
import utils,build_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as python_random
from keras import backend as K

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/nilearn/__init__.py:68: FutureWarning: Python 3.5 support is deprecated and will be removed in a future release. Consider switching to Python 3.6 or 3.7
  _python_deprecation_warnings()


In [2]:
df = pd.read_excel('/home/ubuntu/hacking/projects/jan2021/deep-mediation/results/simulation-2-results/simulation-2-test-image-size-8-svr.xlsx')

In [3]:
df.tail()

,parameters,alpha0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100
97,95,-0.00444384,-0.00452183,-0.00453118,-0.00453222,-0.00453228,-0.00453217,-0.00453236,-0.00453219,-0.00453236,...,4.94775,4.94776,4.94774,4.94779,4.9477,4.94771,4.94769,4.94775,4.94775,4.94771
98,96,0.00845891,0.00837947,0.00837412,0.0083739,0.00837387,0.00837353,0.00837353,0.00837393,0.00837369,...,5.06119,5.06117,5.06118,5.06118,5.06113,5.06116,5.06117,5.06118,5.06117,5.06112
99,97,-0.00437208,-0.00438557,-0.0043892,-0.00438908,-0.00438928,-0.00438894,-0.00438902,-0.00438924,-0.00438928,...,4.98601,4.98596,4.98597,4.98599,4.98599,4.98597,4.98597,4.98597,4.98596,4.98597
100,98,-0.00840623,-0.00849412,-0.0085082,-0.00850948,-0.00850956,-0.00850937,-0.00850939,-0.00850939,-0.00850953,...,5.02038,5.02038,5.02035,5.02037,5.02036,5.02039,5.02039,5.02032,5.02041,5.02038
101,99,-0.00636789,-0.00633389,-0.00633434,-0.00633444,-0.00633433,-0.00633425,-0.00633436,-0.00633445,-0.00633443,...,5.06025,5.06027,5.06024,5.06023,5.06022,5.06026,5.06024,5.06023,5.06023,5.06023


In [4]:
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

epochs = 1000
batch_size = 4
algo = 'svr' #options are [svr,shallow,deep] 
patience = 50
result_path = '/home/ubuntu/hacking/projects/jan2021/deep-mediation/results/simulation-2-results'
if not os.path.isdir(result_path):
    os.makedirs(result_path)
num_subs = 1000
num_iters = 20
num_runs = 100
new_shape = [[32,32],[64,64]]

In [ ]:
for n in new_shape:
    if n[0]==32:
        start = 30
    else:
        start = 0
    train_params_df = utils.create_empty_df(num_runs,num_iters)
    test_params_df = utils.create_empty_df(num_runs,num_iters)
    for runs in range(start,num_runs):
        K.clear_session()
        print("Running simulation for num subs %s starting with %s runs"%(n,runs+1))
        # simulate the dataset 
        df_train, df_test, X_train, X_test = utils.simulate_dataset(2*num_subs,resize= True,
                                          new_shape=n,visualize=False,algo=algo,alpha=0.2)
        # create a model 
        if algo == 'svr':
            model = build_model.create_svr_model(X_train,df_train.m)
        elif algo== 'shallow':
            input_shape = (new_shape[0],new_shape[1]*4,1)
            model = build_model.create_shallow_model2D(input_shape=input_shape)
        else:
            input_shape = (new_shape[0],new_shape[1]*4,1)
            model = build_model.create_deep_model2D(input_shape=input_shape)
        suffix= '-image-size-'+str(n[0])
        train_params_df,test_params_df,z_train = utils.simulate_mediation(df_train,df_test,X_train,X_test,
                            train_params_df,test_params_df,model,suffix,n_runs=runs,batchSize=batch_size,
                            nEpochs=epochs,iterations=num_iters,pat=patience,output_path=result_path,
                            use_model=None,algo=algo)

    train_params_df.to_excel(os.path.join(result_path,'simulation-2-train'+suffix+'-'+algo+'-1.xlsx'))
    test_params_df.to_excel(os.path.join(result_path,'simulation-2-test'+suffix+'-'+algo+'-1.xlsx'))

Running simulation for num subs [32, 32] starting with 31 runs
Dataset is randomly split into 50.0% test dataset and 50.0% train dataset
Total number of subjects in training dataset: 1000
Total number of subjects in testing dataset: 1000
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   32.8s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   35.5s finished


Using intial random model parameters
Starting iteration... 1
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   32.3s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   35.2s finished


Starting iteration... 2
Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:  1.0min remaining:    4.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.0min finished


Starting iteration... 10
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   43.5s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   46.3s finished


Starting iteration... 11
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


In [ ]:
import os,sys
import tensorflow as tf
# import importlib
sys.path.append('/home/ubuntu/hacking/projects/deep-mediation/nov2020')
import auxiliaryfunctions
# importlib.reload(auxiliaryfunctions)
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
from scipy.stats import zscore,norm
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as python_random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error,r2_score

# from datagenerator import dataGenerator
from keras import backend as K

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD

In [ ]:
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

early_stopping = True
epochs = 1000
batch_size = 4
use_dynamic_LR=False
# optimizer='Adam'
algo = 'svr' #['shallow','deep','svr'] 
patience = 50
result_path = '/home/ubuntu/hacking/projects/deep-mediation/nov2020/results/simulation-2-results'
num_subs = 1000
num_iters = 20
num_runs = 100
new_shape = [[64,64]]

In [ ]:
def make_plot(z,m,d,iter):
    plt.figure(figsize=(20,5))
    plt.subplot(1, 4, 1)
    plot_distribution(z,m,labels=['z','m'])

    plt.subplot(1, 4, 2)
    plot_distribution(z,d,labels=['z','d'])

    plt.subplot(1, 4, 3)
    plot_scatter(z,d,labels= ['z','d'])

    plt.subplot(1, 4, 4)
    plot_scatter(z,m,labels= ['z','m'])
    # plt.savefig(os.path.join(result_path,'result'+ str(iter) +'.png'),dpi=300)
    plt.show()

def plot_distribution(z,m,labels):
    sns.distplot(z, hist=True, rug=True,label=labels[0]);
    sns.distplot(m, hist=True, rug=True,label=labels[1]);
    plt.legend()

def plot_scatter(y_test,y_pred,labels):
    r = np.corrcoef(y_pred,y_test)[0,1]
    r = round(r,3)
    plt.scatter(y_test,y_pred)
    plt.xlabel(labels[0])
    plt.ylabel(labels[1])
    plt.title('r=%s'%r)

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

def plot_error(ax,values,title,y_lab,num_iterations):
    ax.plot(values)
    ax.set_xlim([0,num_iterations])
    ax.set_xlabel('#iterations')
    ax.set_ylabel(y_lab)
    ax.set_title(title)

def create_empty_df(num_runs,num_iters):
    # Creates an empty dataFrame
    a = np.empty((num_runs,num_iters))
    a[:] = np.nan
    dataFrame = None
    parameters = ['alpha0', 'beta0','alpha','beta','gamma']
    for params in parameters:
        iter = ['iter_'+str(i) for i in range(num_iters)]
        pdindex = pd.MultiIndex.from_product([[params], iter],
                                             names=['parameters', 'runs']) 
        frame = pd.DataFrame(a, columns = pdindex,index = range(0,num_runs))
        dataFrame = pd.concat([dataFrame,frame],axis=1)
    return dataFrame

def simulate_mediation(df,M,params_df,model,algo,num_subs,batch_size,epochs,n,n_runs,iterations):
    """
    """
    early_stopping=True
    # Initialize z with some phi
    print("Using intial random model parameters")
#     if algo == 'svr':
#         z = svr.predict(M.flatten())
    z = model.predict(M) 
    try:
        z = np.concatenate(z)
    except:
        pass
    # plot_scatter(z,df.m,labels=['z','m'])
    
    for i in range(0,iterations):
        print('Starting iteration... %s'%(i+1))
        #Check for sign change
        if np.corrcoef(z,df.Y)[0,1] < 0 :
            z = z*(-1)
        # Check for scaling
        z= zscore(z)

        lm = smf.ols(formula='z ~ X', data=df).fit()
        alpha0 = lm.params.loc['Intercept']
        alph = lm.params.loc['X'] 

        lm = smf.ols(formula='Y ~ z + X', data=df).fit()
        beta0 = lm.params.Intercept 
        bet = lm.params.z
        gam = lm.params.X 
        resid_std = np.std(lm.resid)
        e = df.Y - beta0 - (df.X*gam)
        h = alpha0 + df.X*alph
        d = (((bet*e)+h)/((bet**2)+1))
#         d = zscore(d)

        if algo== 'svr':
            model = auxiliaryfunctions.create_svr_model(M,d)
            z = model.predict(M)
        else:
            adam = Adam(lr=0.01, decay=0.03)
            model.compile(loss='mean_absolute_error',optimizer=adam)
            if early_stopping:
                es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
                history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=[es],
                                  shuffle=True,validation_split = 0.3)
            else:
                history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,
                              shuffle=True,validation_split = 0.3)
            z = model.predict(M)
       
        params_df.loc[n_runs]['alpha0','iter_'+str(i)]=alpha0
        params_df.loc[n_runs]['beta0','iter_'+str(i)]=beta0
        params_df.loc[n_runs]['beta','iter_'+str(i)]=bet
        params_df.loc[n_runs]['gamma','iter_'+str(i)]=gam
        params_df.loc[n_runs]['alpha','iter_'+str(i)]=alph

        try:
            z = np.concatenate(z)
        except:
            pass
       
    return params_df,z


In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

for n in new_shape:
    start = 68
    output_df = create_empty_df(num_runs,num_iters)
    for runs in range(start,num_runs):
        K.clear_session()
        print("Running simulation for image shape %s starting with %s runs"%(n,runs+1))
    # simulate the dataset
        df,mediator,z = auxiliaryfunctions.simulate_dataset(num_subs,resize= True,
                                          new_shape=n,algo=algo,visualize=False,alpha=0.2)
        # create a model 
        if algo == 'svr':
            model = auxiliaryfunctions.create_svr_model(mediator,df.m)
        elif algo== 'shallow':
            input_shape = (n[0],n[1]*4,1)
            model = auxiliaryfunctions.create_shallow_model2D(input_shape=input_shape)
        else:
            input_shape = (n[0],n[1]*4,1)
            model = auxiliaryfunctions.create_deep_model2D(input_shape=input_shape)

        params_df,z_final = simulate_mediation(df,mediator,output_df,
                                      model,algo,n,batch_size,epochs,
                                      n,runs,num_iters)
        params_df.to_excel(os.path.join(result_path,'simulation-2-image-size-'+str(n[0])+'-'+algo+'-part-1.xlsx'))

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 3740